In [1]:
# Importing libraries

%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Reading data

data = pd.read_csv("Monthly_data_cmo.csv")
data.head()

,APMC,Commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name,state_name
0,Ahmednagar,Bajri,2015,April,79,1406,1538,1463,2015-04,Ahmadnagar,Maharashtra
1,Ahmednagar,Bajri,2016,April,106,1788,1925,1875,2016-04,Ahmadnagar,Maharashtra
2,Ahmednagar,Wheat(Husked),2015,April,1253,1572,1890,1731,2015-04,Ahmadnagar,Maharashtra
3,Ahmednagar,Wheat(Husked),2016,April,387,1750,2220,1999,2016-04,Ahmadnagar,Maharashtra
4,Ahmednagar,Sorgum(Jawar),2015,April,3825,1600,2200,1900,2015-04,Ahmadnagar,Maharashtra


In [3]:
data.drop(["state_name"], axis = 1, inplace =True)
data.sort_values(by = 'date', inplace = True)

# Data Manipulation

-  Removing Duplicate/Erroneous Records

In [4]:
data.Commodity.unique()

array(['Sorgum(Jawar)', 'Ladies Finger', 'Potato', 'Ginger (Fresh)',
       'Ground Nut Pods (Dry)', 'Onion', 'Brinjal', 'Bottle Gourd',
       'Ridge Gourd', 'Bitter Gourd', 'Cucumber', 'Maize',
       'Wheat(Husked)', 'Ghosali(Bhaji)', 'Jaggery', 'Bajri', 'Flower',
       'Ghevda', 'Green Chilli', 'Garlic', 'Green-Peas', 'Tomato',
       'Shevga', 'Cluster Bean', 'Sweet Potato', 'Capsicum', 'Cabbage',
       'Pavtta', 'Chavli (Shenga )', 'Green Gram', 'Gram', 'Ghevda Seed',
       'Snake Gourd', 'Paddy-Unhusked', 'Black Gram', 'Soybean',
       'Sunflower', 'Sesamum', 'Turmeric', 'Pigeon Pea (Tur)',
       'Rice(Paddy-Hus)', 'Lang', 'Linseed', 'Coriander (Dry)',
       'Chillies(Red)', 'Mustard', 'Pavata', 'Squash Gourd',
       'Other Vegetables', 'Cowpea', 'Pomegranate', 'Horse Gram', 'Matki',
       'Apple', 'Mosambi', 'Banana', 'Chikoo', 'Castor Seed',
       'Elephant Root', 'Small Gourd', 'Ginger (Dry)', 'Cummin',
       'Cashewnuts', 'Fenugreek', 'Tamarind', 'Custard Apple', '

**As it can be observed, there are many commodities with same names but are recorded as 2 different commodities pertaining to use of capitals.** <br/>
As for an example: *'Paddy-Unhusked' and 'PADDY-UNHUSKED', 'Bajri' and 'BAJRI', 'Garlic' and 'GARLIC'*  <br/>
**So, I convert all the commodity names to lowercase and merge the records of similar commodities. <br/>**

In [5]:
# Converting all Commodity names to lowercase in order to avoid duplicate entries

data["Commodity"] = [x.lower() for x in data["Commodity"]]

**Without loss of generality, I merge the commodities which register under a single name to one particular APMC on the same date with same minimum, maximum and modal prices.**

In [6]:
# Records of all the commodities which have been registered under same names.

data[data.duplicated(['APMC', 'Commodity', 'date', 'min_price', 'max_price', 'modal_price'], keep=False)]

,APMC,Commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name
50613,Satara,garlic,2014,September,63,2000,5000,3500,2014-09,Satara
50590,Satara,garlic,2014,September,125,2000,5000,3500,2014-09,Satara
50294,Satara,garlic,2014,October,42,2000,5000,3500,2014-10,Satara
50271,Satara,garlic,2014,October,126,2000,5000,3500,2014-10,Satara
40846,Pune-Pimpri,garlic,2015,August,27,6000,6000,6000,2015-08,Pune
40828,Pune-Pimpri,garlic,2015,August,36,6000,6000,6000,2015-08,Pune
42146,Pune-Pimpri,garlic,2016,January,3,6000,6000,6000,2016-01,Pune
42167,Pune-Pimpri,garlic,2016,January,24,6000,6000,6000,2016-01,Pune
41723,Pune-Pimpri,garlic,2016,February,18,6000,6000,6000,2016-02,Pune
41740,Pune-Pimpri,garlic,2016,February,24,6000,6000,6000,2016-02,Pune


**As, it can be observed, registering commodities with all capitals and with Camel case has caused duplicate/erroneous entries.** <br/>
<br/>

In [7]:
data[data.duplicated(['APMC', 'Commodity', 'date', 'min_price', 'max_price', 'modal_price'], keep=False)].shape

(20, 10)


**So, there are 20 such records which need to be merged** <br/>
<br/>
**To merge these records, I add the total arrival in quintals and assign it to a single record and delete the second one.**

In [8]:
# Obtaining the list of indexes which contain duplicate entries

j = np.array(data.loc[data.duplicated(['APMC', 'Commodity', 'date', 'min_price', 'max_price', 'modal_price'], keep=False)].index)
d = []

In [9]:
# Adding and assigning total arrivals in quintal to one record

for i in range(0, len(j)-1, 2):
    data.loc[j[i], "arrivals_in_qtl"] = data.loc[j[i]]["arrivals_in_qtl"] + data.loc[j[i+1]]["arrivals_in_qtl"]
    d.append(j[i+1])

data.loc[j]

,APMC,Commodity,Year,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name
50613,Satara,garlic,2014,September,188,2000,5000,3500,2014-09,Satara
50590,Satara,garlic,2014,September,125,2000,5000,3500,2014-09,Satara
50294,Satara,garlic,2014,October,168,2000,5000,3500,2014-10,Satara
50271,Satara,garlic,2014,October,126,2000,5000,3500,2014-10,Satara
40846,Pune-Pimpri,garlic,2015,August,63,6000,6000,6000,2015-08,Pune
40828,Pune-Pimpri,garlic,2015,August,36,6000,6000,6000,2015-08,Pune
42146,Pune-Pimpri,garlic,2016,January,27,6000,6000,6000,2016-01,Pune
42167,Pune-Pimpri,garlic,2016,January,24,6000,6000,6000,2016-01,Pune
41723,Pune-Pimpri,garlic,2016,February,42,6000,6000,6000,2016-02,Pune
41740,Pune-Pimpri,garlic,2016,February,24,6000,6000,6000,2016-02,Pune


**As it can be observed, values of alternate records starting from first one have been updated**<br/>

**Size of data before dropping extra rows**

In [10]:
data.shape

(62429, 10)

In [11]:
data.drop(d, inplace = True)

**Size of data after dropping extra rows**

In [12]:
data.shape

(62419, 10)

**Therefore, 10 duplicate/erroneous records have been dropped**

# Comparison with Minimum Support Price
 - Comparing the commodities whose MS Price is available in data given with their Minimum Price, Maximum Price and Modal Price

In [13]:
# Reading file in which MSPs are given

file = pd.read_csv("CMO_MSP_Mandi.csv")
file.head()

,commodity,year,Type,msprice,msp_filter
0,PADDY-UNHUSKED,2012,Kharif Crops,1250.0,1
1,RICE(PADDY-HUS),2012,Kharif Crops,1280.0,1
2,Jowar_Hybrid,2012,Kharif Crops,1500.0,1
3,SORGUM(JAWAR),2012,Kharif Crops,1520.0,1
4,BAJRI,2012,Kharif Crops,1175.0,1


In [14]:
file.shape

(155, 5)

In [15]:
data.Year.unique()

array([2014, 2015, 2016], dtype=int64)

In [16]:
file.year.unique()

array([2012, 2013, 2014, 2015, 2016], dtype=int64)

**As it can be observed, 'file' contains MSPs for years 2012 to 2016, but Min, Max and Modal prices are known for years 2014 to 2016 only.** <br/>
**So, rows having MSPs for years earlier than 2014 can be dropped**

In [17]:
file.drop(file[file["year"] < 2014].index, inplace = True)
file.shape

(93, 5)

**More than 50 records dropped**

In [18]:
# Checking for null values in dataframe file
    
file.isnull().sum()

commodity      0
year           0
Type           0
msprice       10
msp_filter     0
dtype: int64

**10 records where MSP is missing** <br/>
**So we remove these records**

In [19]:
file[file["msprice"].isnull()]

,commodity,year,Type,msprice,msp_filter
107,Soyabean_Black,2015,Kharif Crops,NaN,1
138,Soyabean_Black,2016,Kharif Crops,NaN,1
142,WHEAT(HUSKED),2016,Rabi Crops,NaN,1
143,WHEAT(UNHUSKED),2016,Rabi Crops,NaN,1
144,BARLI,2016,Rabi Crops,NaN,1
145,Gram_Yellow,2016,Rabi Crops,NaN,1
146,Masur_Yellow,2016,Rabi Crops,NaN,1
147,MUSTARD,2016,Rabi Crops,NaN,1
148,SAFFLOWER,2016,Rabi Crops,NaN,1
149,Toria_Yellow,2016,Rabi Crops,NaN,1


In [20]:
file.dropna(inplace = True)

In [21]:
file.shape

(83, 5)

**10 records dropped.**

In [22]:
file.msp_filter.unique()

array([1], dtype=int64)

MSP Filter is same for all records, therefore, it can be dropped.

In [23]:
file.drop(["msp_filter"], axis = 1, inplace =True)

### Now, to compare the prices, I create a new dataframe which contains data only about the commodities for which all 4 prices(Min, Max, Modal, MSP) are known. <br/>
**So, I perform an inner join on the exisitng dataframes**

In [24]:
# Converting all Commodity names to lowercase

file["commodity"] = [x.lower() for x in file["commodity"]]

In [25]:
file[file.duplicated(['commodity', 'year', 'Type'], keep=False)]

,commodity,year,Type,msprice


Therefore, no duplicate records found

In [26]:
# Renaming column names to be consistent across both files

file.columns = ['Commodity', 'Year', 'Type', 'ms_price']

In [27]:
combined = pd.merge(file, data, on = ['Commodity', 'Year'], how = 'inner')

In [28]:
combined.head()

,Commodity,Year,Type,ms_price,APMC,Month,arrivals_in_qtl,min_price,max_price,modal_price,date,district_name
0,paddy-unhusked,2014,Kharif Crops,1360.0,Shahapur,September,49,1415,1521,1488,2014-09,Thane
1,paddy-unhusked,2014,Kharif Crops,1360.0,Vasai,September,80,1500,1800,1650,2014-09,Thane
2,paddy-unhusked,2014,Kharif Crops,1360.0,Ghoti,September,30,2200,2200,2200,2014-09,Nasik
3,paddy-unhusked,2014,Kharif Crops,1360.0,Murbad,September,154,1700,1867,1600,2014-09,Thane
4,paddy-unhusked,2014,Kharif Crops,1360.0,Ausa,September,11,851,1106,986,2014-09,Latur


In [29]:
# Organising dataframe

combined.drop(["Month", "district_name"], axis = 1, inplace =True)
combined= combined[['Commodity', 'APMC', 'Year', 'date', 'min_price', 'modal_price', 'max_price', 'ms_price', 'arrivals_in_qtl']]

In [30]:
combined.head()

,Commodity,APMC,Year,date,min_price,modal_price,max_price,ms_price,arrivals_in_qtl
0,paddy-unhusked,Shahapur,2014,2014-09,1415,1488,1521,1360.0,49
1,paddy-unhusked,Vasai,2014,2014-09,1500,1650,1800,1360.0,80
2,paddy-unhusked,Ghoti,2014,2014-09,2200,2200,2200,1360.0,30
3,paddy-unhusked,Murbad,2014,2014-09,1700,1600,1867,1360.0,154
4,paddy-unhusked,Ausa,2014,2014-09,851,986,1106,1360.0,11


### Observations across clusters formed by Commodities and Years

In [31]:
g =  combined.groupby(["Commodity", "APMC"])

In [ ]:
# As this cell generates a lot images(graphs), size of the file exceeds the Github upload limit of 25 MB. So I am not executing the cell.
# However, the code is completely working and you may try it out on your PC


# Plotting Minimum, Maximum, Modal and MS prices for each cluster: APMC and Commodity

for k, df in g:
    print(k)
    plt.figure()
    plt.plot( 'date', 'min_price', data=df, marker='+', color='blue', label = "Minimum Price")
    plt.plot( 'date', 'max_price', data=df, marker='s', color='green', label = "Maximum Price")
    plt.plot( 'date', 'modal_price', data=df, marker='o', color='yellow', label="Modal Price")
    plt.plot( 'date', 'ms_price', data=df, marker='*', color='red', label="Minimum Support Price")
    plt.title(k)
    plt.legend()

# Outliers Detection

 -  I use two methods for outlier detection: <br/>
     1. IQR (points 1.5*IQR above and below Q3 and Q1 respectively)
     2. Z Scores (points more than Z Score of 3.5)
    
Functions of these methods were executed on all the three prices of the dataset but not outliers were found.

In [32]:
 # Outlier detection function using Z Scores method

def outliers_z_score(ys):
    threshold = 3.5
    median_y = np.median(ys)
    median_absolute_deviation_y = np.median([np.abs(y - median_y) for y in ys])
    modified_z_scores = [0.6745 * (y - median_y) / median_absolute_deviation_y for y in ys]
    return np.where(np.abs(modified_z_scores) > threshold)

In [33]:
# Using Z Scores method

for h, df in data.groupby(["APMC", "Commodity"]):
    a = outliers_z_score(df["modal_price"])
    if len(a) > 1:
        print(h)
        print(df["modal_price"])
        print(a)
    
    a = outliers_z_score(df["max_price"])
    if len(a) > 1:
        print(h)
        print(df["max_price"])
        print(a)
    
    a = outliers_z_score(df["min_price"])
    if len(a) > 1:
        print(h)
        print(df["min_price"])
        print(a)    

H:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
H:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater
  
H:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


In [34]:
# Outlier detection function using IQR Method

def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where((ys > upper_bound) | (ys < lower_bound))

In [35]:
# Using IQR method

for h, df in data.groupby(["APMC", "Commodity"]):
    
    a = outliers_iqr(df["modal_price"])
    if len(a) > 1:
        print(h)
        print(df["modal_price"])
        print(a)
    
    a = outliers_iqr(df["max_price"])
    if len(a) > 1:
        print(h)
        print(df["max_price"])
        print(a)
    
    a = outliers_iqr(df["min_price"])
    if len(a) > 1:
        print(h)
        print(df["min_price"])
        print(a)    

**Therefore, no outliers in any of the three prices detected**

# Trend and Seasonality

In [ ]:
# Detecting trends by using Rolling Mean (Window size of 12 as the data is given on monthly basis)

for k, df in data.groupby(["APMC", "Commodity"]):
    print(k)
    mi = df["min_price"].rolling(12).mean()
    ma = df["max_price"].rolling(12).mean()
    mo = df["modal_price"].rolling(12).mean()
    dt = df['date']
    plt.figure()
    plt.plot(dt, mi, marker='+', color='blue', label = "Minimum Price")
    plt.plot(dt, ma, marker='s', color='green', label = "Maximum Price")
    plt.plot(dt, mo, marker='o', color='yellow', label="Modal Price")
    plt.xlabel('Year', fontsize=20)

**As most of clusters in the data do not have more than or equal to 12 records, the rolling mean method fails.**

# Fluctuations

**For every month in each year, maximum fluctuation for every commodity is calculated and the maximum of all these maximums is taken as the maximum fluctuation of that particualr month in that year**

In [36]:
ind = []

g = data.groupby(["Year", "Month"])

for k, df in g:
    print(k)
    f = []
    v = df.groupby(["Commodity"])
    for h, d in v:

        ind.append(d[d.modal_price == d.modal_price.max()].index)
        f.append(d.modal_price.max() - d.modal_price.min())
     
    i = f.index(max(f))
    print(data.loc[ind[i]])

(2014, 'December')
             APMC      Commodity  Year     Month  arrivals_in_qtl  min_price  \
22873  Gadhinglaj  chillies(red)  2014  December              292       5444   

       max_price  modal_price     date district_name  
22873      42556        18967  2014-12      Kolhapur  
(2014, 'November')
       APMC Commodity  Year     Month  arrivals_in_qtl  min_price  max_price  \
41096  Pune    chikoo  2014  December                7       4500       5000   

       modal_price     date district_name  
41096         4750  2014-12          Pune  
(2014, 'October')
             APMC      Commodity  Year     Month  arrivals_in_qtl  min_price  \
22873  Gadhinglaj  chillies(red)  2014  December              292       5444   

       max_price  modal_price     date district_name  
22873      42556        18967  2014-12      Kolhapur  
(2014, 'September')
                       APMC    Commodity  Year     Month  arrivals_in_qtl  \
40976  Junnar (Narayangaon)  coriander    2014  December

**Here, atfirst I consider each month for all the three years. Then, for each commodity, I calculate the fluctuation(maximum price for that month and year - minimum price for that month and year), and then select the commodity with maximum fluctuation for that month and year.**

In [38]:
# Saving dataframe as a CSV file.

data.to_csv("Monthly_data_cmo", sep='\t', encoding='utf-8')